<a href="https://colab.research.google.com/github/SabrinaGalvanS/RepositorioGit/blob/master/Practica_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo Best-First Search para problemas de búsqueda

El algoritmo de Best-First Search es un algoritmo simple basado en un árbol de búsqueda. Este algoritmo toma como entrada un problema de búsqueda y construye un árbol de búsqueda a partir de este.

Para definirlo, presentaremos en primer lugar los elementos necesarios para definir nuestro problema de búsqueda.

## Problema de búsqueda - Puzzle 8

Definiremos el problema del mundo de la aspiradora, para esto recordemos que un problema de búsqueda cuenta con los siguientes elementos:

1. Conjunto de estados
1. Estado inicial
1. Estados finales
1. Acciones
1. Modelo de transición
1. Función de costo

In [ ]:
import numpy as np

## 1. Espacio de estados cvc

Podemos representar a cada ficha con su respectiva posición como una lista de Python, donde cada índice corresponde al lugar de la ficha y cada valor corresponde a la ficha misma. 


Los índices-posiciones se verán de la siguiente manera:

|   |   |   |
|---|---|---|
| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

El espacio lo podemos representar con un *0*

In [ ]:
#conjunto de estados
e = np.random.choice(range(9), replace=False, size=9)
e

array([1, 3, 5, 0, 4, 2, 8, 7, 6])

|   |   |   |
|---|---|---|
| 1 | 3 | 5 |
|   | 4 | 2 |
| 8 | 7 | 6 |

## 2. Estado Inicial 

In [ ]:
estado_inicial = [1,0,2,6,3,4,7,5,8]

|   |   |   |
|---|---|---|
| 1 |   | 2 |
| 6 | 3 | 4 |
| 7 | 5 | 8 |

## 3. Estado objetivo 

In [ ]:
estado_meta = [0,1,2,3,4,5,6,7,8]

|   |   |   |
|---|---|---|
|   | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

## 4. Acciones disponibles 

Podemos especificar las acciones a tomar como las duplas (lugar de pieza a mover, dirección del movimiento)


Por ejemplo:

|   |   |   |
|---|---|---|
|   | 8 | 2 |
| 5 | 4 | 3 |
| 6 | 7 | 1 |

Para este estado, las acciones posibles son:

                                    {(1, izquierda), (3, arriba)}
                                    
Pero si lo pensamos, en nuestra representación del ambiente, el movimiento de la acción no es esencial, porque una vez escogido el lugar de la pieza, el movimiento ya queda determinado: no podemos hacer otra cosa más que **intercambiar el espacio por la ficha**.

Sin embargo, vamos a dejar la acción por conveniencia, además de incluir el estado donde estaba la posición disponible (solo para ahorrar código)

In [ ]:
def accion(estado):
    if estado[0]==0:
        return [(1,'left',0), (3,'up',0)]
    elif estado[1]==0:
        return [(0,'right',1), (2,'left',1),(4,'up',1)]
    elif estado[2]==0:
        return [(1,'right',2), (5,'up',2)]
    elif estado[3]==0:
        return [(0,'down',3), (4,'left',3), (6,'up',3)]
    elif estado[4]==0:
        return [(1,'down',4), (3,'right',4), (5,'left',4), (7,'up',4)]
    elif estado[5]==0:
        return [(2,'down',5), (4,'right',5), (8,'up',5)]
    elif estado[6]==0:
        return [(3,'down',6), (7,'left',6)]
    elif estado[7]==0:
        return [(4,'down',7), (6,'right',7), (8,'left',7)]
    else:
        return [(5,'down',8), (7,'right',8)]
accion([1, 3, 5, 0, 4, 2, 8, 7, 6])

[(0, 'down', 3), (4, 'left', 3), (6, 'up', 3)]

## 5. Modelo de transición

Aquí simplemente tenemos que reflejar ese intercambio de espacio-ficha:

In [ ]:
def transicion(estado, accion):

    lugar_pieza_a_mover = accion[0] #el lugar seleccionado de la ficha a mover
    lugar_disponible = accion[2] #donde está el espacio
    pieza = estado[lugar_pieza_a_mover] #el valor de la ficha
    
    nuevo_estado = estado.copy() #nuevo estado para hacer el reemplazo
    
    nuevo_estado[lugar_pieza_a_mover] = 0 #reemplazando la ficha por espacio
    nuevo_estado[lugar_disponible] = pieza #reemplazando espacio por ficha
    return nuevo_estado
transicion([1, 3, 5, 0, 4, 2, 8, 7, 6], accion([1, 3, 5, 0, 4, 2, 8, 7, 6])[1])

#[1, 3, 5, 
# 0, 4, 2, 
# 8, 7, 6]
#
#[1, 3, 5, 
# 4, 0, 2, 
# 8, 7, 6]

[1, 3, 5, 4, 0, 2, 8, 7, 6]

## 6. Función de costos

Asumiremos que cada acción tiene un costo 1, por lo que

c(s',a,s) = c = 1

## Búsqueda
Ahora ya tenemos todas las piezas necesarias para implementar nuestro algoritmo de búsqueda

In [ ]:
class Node():
    def __init__(self):
        self.state = None
        self.parent = None
        self.action = None
        self.cost = 0

In [ ]:
def expand(node):
    #frontera = []
    #for accion_ in accion(estado):
    #    frontera.append( transicion(estado,accion_)  )
    #return frontera
    estado = node.state
    for action in accion(estado):
        #Ejecuta la acción
        new_state = transicion(estado, action)
        #Guarda el costo de la acción
        cost = node.cost + 1 #siempre cuesta 1 nuestra accion
        
        #Genera un nuevo nodo
        new_node = Node()
        new_node.state = new_state
        new_node.parent = node
        new_node.action = action[1] #guardamos la "accion"
        new_node.cost = cost
        yield new_node #va apendeando a un iterable 
test_node = Node()
test_node.state = [1,0,2,6,3,4,7,5,8]
list(expand(test_node))[0].action

'right'

In [ ]:
[1,0,2,
 6,3,4,
 7,5,8]

In [ ]:
accion(test_node.state)

[(0, 'right', 1), (2, 'left', 1), (4, 'up', 1)]

In [ ]:
estado_inicial = [1,0,2,6,3,4,7,5,8]
estado_inicial = [1,0,2,3,4,5,6,7,8]
estado_meta = [0,1,2,3,4,5,6,7,8]

nodo_inicial = Node()
nodo_inicial.state = estado_inicial #imponiendo el estado inicial

nodos = [] #camino de nodos
frontier = [nodo_inicial] #poblamos la frontera con el nodo inicial
reached = {str(estado_inicial):nodo_inicial}

def search():
    while len(frontier)>0:
        nodo = frontier.pop(0) #removiendo primer elemento de la lista

        nodos.append(nodo) #guardamos el nodo alcanzado

        if nodo.state == estado_meta:
            #break
            return nodos

        #Expande el nodo actual
        for child in expand(nodo):
            #Guarda estado de los hijos
            state = child.state
            #Guarda los hijos no alcanzados o que tengan menor costo
            if str(state) not in reached.keys() or child.cost < reached[str(state)].cost:
                reached[str(state)] = child
                frontier.append(child)    
    return "No se ha logrado llegar a un estado final."
solucion = search()
len(solucion)

2